### Tools

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain.chat_models import init_chat_model
model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Hello")
response

AIMessage(content='<think>\nOkay, the user sent "Hello". I should respond in a friendly and welcoming manner. Let me make sure to acknowledge their greeting and offer assistance. Maybe something like, "Hello! How can I assist you today?" That should be appropriate. I need to keep it open-ended so they feel comfortable asking for help with whatever they need. No need for any markdown, just a simple, warm response.\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 9, 'total_tokens': 106, 'completion_time': 0.194085928, 'completion_tokens_details': None, 'prompt_time': 0.000284775, 'prompt_tokens_details': None, 'queue_time': 0.052141155, 'total_time': 0.194370703}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb6e4-3654-7b80-9311-fe6b9aa21534-0', tool

In [6]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"

model_with_tools = model.bind_tools([get_weather])
response = model_with_tools.invoke("What's the weather in Pune?")
response

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Pune. I need to use the get_weather function. The function requires a location parameter. Pune is a city in India, so I should specify that as the location. I\'ll call the function with "Pune, India" to ensure it\'s clear. Let me make sure the JSON is formatted correctly with the location parameter. Yep, that\'s all set.\n', 'tool_calls': [{'id': 's56fftc4q', 'function': {'arguments': '{"location":"Pune, India"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 153, 'total_tokens': 263, 'completion_time': 0.21467469, 'completion_tokens_details': {'reasoning_tokens': 83}, 'prompt_time': 0.006030412, 'prompt_tokens_details': None, 'queue_time': 0.159517588, 'total_time': 0.220705102}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_c

In [7]:
response.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Pune, India'},
  'id': 's56fftc4q',
  'type': 'tool_call'}]

### Tool Execution loop

In [10]:
messages = [{"role": "user", "content": "What's the weather in Pune?"}]
ai_message = model_with_tools.invoke(messages)
messages.append(ai_message)

for tool_call in ai_message.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

final_response = model_with_tools.invoke(messages)
print(final_response.text)

The current weather in Pune is sunny. Let me know if you need more details! ☀️


In [11]:
model_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x10b347a50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10b40bed0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather at a location', 'parameters': {'properties': {'location': {'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}]}, config={}, config_factories=[])